In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize

In [44]:
df = pd.read_csv('../data/aden_golfe.csv',sep=';')

In [45]:
max_quinzaine=24
max_area_lat=12
max_area_lon=31
number_grid=(max_area_lat+1)*(max_area_lon+1)
print number_grid

416


In [46]:
def monthCorrect(month):
    month=str(int(month))
    if len(month)==1:
        return '0'+month
    else:
        return month

In [47]:
df["mois (0)"]=df.apply(lambda row: monthCorrect(row['mois (0)']), axis=1)
df["jour (0)"]=df.apply(lambda row: monthCorrect(row['jour (0)']), axis=1)
df["date"]=df.apply(lambda row: str(row["Annee (0)"]) +"-"+ str(row["mois (0)"]), axis=1)

# Version 1 : group by years, month, area

In [48]:
df['lat_area'] = df['lat_area'].astype(int)
df['lon_area'] = df['lon_area'].astype(int)
df['area'] = df.apply(lambda row: str(row['lat_area'])+'_'+str(row['lon_area']), axis=1)

## Creation d'un data frame vide

In [49]:
dfAgregateTotal=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","mois (0)","area"] ).size()}).reset_index()

In [50]:
specifecArea=pd.DataFrame(dfAgregateTotal['area'].str.split('_',1).tolist(),columns = ['lat_area','lon_area'])
dfAgregateTotal=pd.concat([dfAgregateTotal, specifecArea], axis=1)
print dfAgregateTotal.head(20)
print len(dfAgregateTotal)

dfAgregateTotal['mois (0)']=dfAgregateTotal['mois (0)'].astype(int).astype(str)
dfAgregateTotal['lat_area']=dfAgregateTotal['lat_area'].astype(int)
dfAgregateTotal['lon_area']=dfAgregateTotal['lon_area'].astype(int)

    Annee (0) mois (0)  area  count lat_area lon_area
0        2008       02  4_20      2        4       20
1        2008       03  3_21      1        3       21
2        2008       03  4_18      1        4       18
3        2008       03  6_19      1        6       19
4        2008       04  4_16      1        4       16
5        2008       04  4_17      1        4       17
6        2008       04  5_16      1        5       16
7        2008       04  5_18      1        5       18
8        2008       04  6_18      1        6       18
9        2008       04  7_16      1        7       16
10       2008       05  4_16      1        4       16
11       2008       05  4_19      1        4       19
12       2008       05  5_14      1        5       14
13       2008       05  5_16      1        5       16
14       2008       05  5_19      1        5       19
15       2008       05  6_19      1        6       19
16       2008       06  4_14      1        4       14
17       2008       06  4_17

In [51]:
uniqueLat=dfAgregateTotal[["lat_area","lon_area"]].values
unique=pd.Series(zip(uniqueLat[:,0], uniqueLat[:,1])).unique()


print "--------------------------------------------"
list1, list2 = zip(*unique)
array1=np.asarray(list1)
array2=np.asarray(list2)

ind = np.lexsort((array1,array2)) 
uniqueSort=[(array1[i],array2[i]) for i in ind]



print "--------------------------------------------"

list1, list2 = zip(*uniqueSort)
array1_=np.asarray(list1)
array2_=np.asarray(list2)
a = np.column_stack((array1_,array2_))
#print a


--------------------------------------------
--------------------------------------------


In [52]:
uniqueYears=dfAgregateTotal['Annee (0)'].unique()
print "-------------------------------------------------------------------"
ListAreaLat=np.tile(a[:,0],12*len(uniqueYears))
ListAreaLon=np.tile(a[:,1],12*len(uniqueYears))

uniqueMonth=np.linspace(1, 12, num=12)
ColMonth=np.repeat(uniqueMonth,len(a[:,1]))
#print ColMonth[1:100]
ColMonth=np.tile(ColMonth,len(uniqueYears))


Colyears=np.repeat(uniqueYears, 12*len(a[:,1]))
count=np.repeat(0, len(Colyears))


print len(ColMonth)
print len(Colyears)
print len(ListAreaLat)
print len(ListAreaLon)

-------------------------------------------------------------------
6888
6888
6888
6888


## Fill the matrice

In [53]:
#columns = ['Years','2_week', 'lat_area','lon_area','count']
#matriceML=pd.DataFrame(columns=columns)
#matriceML = matriceML.fillna(0)

dic = {'Years': Colyears, 
     'Month': ColMonth, 
     'lat_area': ListAreaLat,
     'lon_area': ListAreaLon,
     'count': count
      }

dfFinal= pd.DataFrame(dic)

dfFinal["Month"]=dfFinal.apply(lambda row: monthCorrect(row['Month']), axis=1)
dfFinal['Month']=dfFinal['Month'].astype(int)
dfFinal['lat_area']=dfFinal['lat_area'].astype(int)
dfFinal['lon_area']=dfFinal['lon_area'].astype(int)

print dfFinal.head(10)
print dfFinal.dtypes
print dfFinal[(dfFinal['Years']==2008) &
                (dfFinal['lat_area']==4) &
                (dfFinal['lon_area']==20 )&
                (dfFinal['Month']==1)]

   Month  Years  count  lat_area  lon_area
0      1   2008      0         8         2
1      1   2008      0         9         2
2      1   2008      0         6         3
3      1   2008      0         7         3
4      1   2008      0         8         3
5      1   2008      0         5         4
6      1   2008      0         6         4
7      1   2008      0         7         4
8      1   2008      0         4         5
9      1   2008      0         5         5
Month       int64
Years       int64
count       int64
lat_area    int64
lon_area    int64
dtype: object
    Month  Years  count  lat_area  lon_area
50      1   2008      0         4        20


In [54]:
for index, row in dfAgregateTotal.iterrows():
    if index%100==0:
        print index
    #print "---------------------------------------------------------"
    #print "len : "+str(len(dfFinal['Years']==row['Annee (0)']))
    
    #print row['Annee (0)'], int(row['mois (0)']), row['count'] ,  row['lat_area'] , row['lon_area'] 
    
    
    idx=dfFinal[(dfFinal['Years']==row['Annee (0)']) &
                (dfFinal['lat_area']==row['lat_area']) &
                (dfFinal['lon_area']==row['lon_area'] )&
                (dfFinal['Month']==int(row["mois (0)"]))]
    
    #print len(idx)
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    #dfFinal.set_value(idx.index.values, 'count',row['count'])
    dfFinal.set_value(idx.index.values, "count",row['count'])

0
100
200
300


In [55]:
print dfFinal['count'].sum()
print len(dfFinal)
print dfFinal[dfFinal['count']!=0].head()

421
6888
     Month  Years  count  lat_area  lon_area
132      2   2008      2         4        20
207      3   2008      1         4        18
212      3   2008      1         6        19
219      3   2008      1         3        21
281      4   2008      1         4        16


## Write matrice in a file

In [56]:
dfFinal.to_csv("../data/aden_golfe_matrix_area.csv",sep=';', encoding='utf-8', index=False)

#  

# Version 2 : group by Date, type attack

In [121]:
dfAgregateTotal2=pd.DataFrame({'count' : df.groupby( [ "date","type d'attaque(0)"] ).size()}).reset_index()
print dfAgregateTotal2.head(10)

      date type d'attaque(0)  count
0  2008-02        Fired upon      1
1  2008-02          Hijacked      1
2  2008-03         Attempted      1
3  2008-03           Boarded      1
4  2008-03        Fired upon      1
5  2008-04         Attempted      1
6  2008-04        Fired upon      3
7  2008-04          Hijacked      2
8  2008-05         Attempted      2
9  2008-05        Fired upon      2


In [123]:
col_years=np.repeat(np.sort(df["Annee (0)"].unique()), max_month)

month_repeat=np.linspace(1, max_month, num=max_month)
month=np.tile(month_repeat, len(unique_years))
col_count=np.repeat(0, len(month))
                               
dic2 = {'Years': col_years,
     'Month': month,
     'Fired upon': col_count,
     'Hijacked': col_count,
      'Attempted': col_count,
     'Boarded': col_count
       }
dfFinal_2 = pd.DataFrame(dic2)
dfFinal_2.head()
dfFinal_2["Month"]=dfFinal_2.apply(lambda row: monthCorrect(row['Month']), axis=1)
dfFinal_2["date"]=dfFinal_2.apply(lambda row: str(row["Years"]) +"-"+ str(row["Month"]), axis=1)
dfFinal_2 = dfFinal_2.drop('Month', 1)                            
dfFinal_2 = dfFinal_2.drop('Years', 1)
dfFinal_2.head()

,Attempted,Boarded,Fired upon,Hijacked,date
0,0,0,0,0,2008-01
1,0,0,0,0,2008-02
2,0,0,0,0,2008-03
3,0,0,0,0,2008-04
4,0,0,0,0,2008-05


## Matrix empty

In [124]:
dfFinal_2.head()

,Attempted,Boarded,Fired upon,Hijacked,date
0,0,0,0,0,2008-01
1,0,0,0,0,2008-02
2,0,0,0,0,2008-03
3,0,0,0,0,2008-04
4,0,0,0,0,2008-05


# Fill matrice

In [129]:
for index, row in dfAgregateTotal2.iterrows():

    #print 'Years '+str(row['Annee (0)']),'2_week : '+str( row['quinzaine_week']),'count : '+str(row['count'])#,'type_attack : '+str(row['type d\'attaque(0)'])

    idx=dfFinal_2[(dfFinal_2['date']==row['date'])]
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    dfFinal_2.set_value(idx.index.values, str(row["type d'attaque(0)"]),row['count'])

In [131]:
#binary=label_binarize(col_type, classes=unique_type)
#dfFinal_2[unique_type[0]]=binary[:,0]
#dfFinal_2[unique_type[1]]=binary[:,1]
#dfFinal_2[unique_type[2]]=binary[:,2]
#dfFinal_2[unique_type[3]]=binary[:,3]

dfFinal_2['date']=dfFinal_2.apply(lambda row: str(row['date'])+'-01', axis=1)
print dfFinal_2

    Attempted  Boarded  Fired upon  Hijacked        date
0           0        0           0         0  2008-01-01
1           0        0           1         1  2008-02-01
2           1        1           1         0  2008-03-01
3           1        0           3         2  2008-04-01
4           2        0           2         2  2008-05-01
5           1        0           1         0  2008-06-01
6           2        0           1         1  2008-07-01
7           0        0           3         6  2008-08-01
8           7        0           6         6  2008-09-01
9           7        0           4         5  2008-10-01
10          5        0           4         7  2008-11-01
11          1        1           5         3  2008-12-01
12          8        0           8         3  2009-01-01
13          4        0           4         1  2009-02-01
14          8        0           4         1  2009-03-01
15          3        0           9         5  2009-04-01
16          8        0         

In [132]:
dfFinal_2.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)